In [ ]:
from doge_admm import dogeADMM, ADMM,fast_ADMM,ORIADMM
from PIL import Image
import numpy as np
from scipy.sparse import lil_matrix
import scipy.sparse
import matplotlib.pyplot as plt
import networkx as nx
from tfgraph import *
import copy
import time


In [ ]:
def add_gaussian_noise(image, mean=0, std=20):
    noise = np.random.normal(mean, std, image.shape) 
    noise[0,:] = 0
    noise[:,0] = 0
    noise[:,-1] = 0
    noise[-1,:] = 0
    noisy_image = image + noise 
    noisy_image = np.clip(noisy_image, 0, 255) 
    return noisy_image

## Monte Carlo


In [ ]:
image_path = 'doge.png' 
image = Image.open(image_path)
gray_image = image.convert('L').resize((64,64))
image = np.array(gray_image)

In [ ]:
t_m = []
loss_m = []
k = 1
B = 2
for _ in range(50):
    image_noise = add_gaussian_noise(image)
    z,Loss,Time = dogeADMM(image_noise,k=k,B=B,l=10,max_iter=100,get_loss_seq=True,threshold=1e-4)
    t_m.append(Time[-1])
    loss_m.append(Loss[-1])

t_m = np.array(t_m)
loss_m = np.array(loss_m)

t_m.mean(),loss_m.mean(),t_m.std(),loss_m.std()

In [ ]:
t_m = []
loss_m = []
k = 1
for _ in range(50):
    image_noise = add_gaussian_noise(image)
    z,Loss,Time = ADMM(image_noise,k,lambd=10,rho=1,max_iter=100,tol=1e-3)
    t_m.append(Time[-1])
    loss_m.append(Loss[-1])

t_m = np.array(t_m)
loss_m = np.array(loss_m)

t_m.mean(),loss_m.mean(),t_m.std(),loss_m.std()

In [ ]:
t_m = []
loss_m = []
iters = []
k = 1
B = 2
for _ in range(50):
    image_noise = add_gaussian_noise(image)
    z,Loss,iter,Time = fast_ADMM(image_noise,k=k,l=10,rho=1,max_iter=100,tol=1e-3)
    t_m.append(Time[-1])
    loss_m.append(Loss[-1])
    iters.append(iter)

t_m = np.array(t_m)
loss_m = np.array(loss_m)
iter = np.array(iters)

t_m.mean(),loss_m.mean(),t_m.std(),loss_m.std(),iters.mean().iters.std()

In [ ]:
t_m = []
loss_m = []
iters = []
k = 1
B = 2
for _ in range(50):
    image_noise = add_gaussian_noise(image)
    z, Loss, iter,Time = ORIADMM(image_noise, rho=1,l=10, max_iter=100, k=k, tol=1e-3)
    t_m.append(Time[-1])
    loss_m.append(Loss[-1])
    iters.append(iter)


t_m = np.array(t_m)
loss_m = np.array(loss_m)
iter_m = np.array(iters)

t_m.mean(),loss_m.mean(),t_m.std(),loss_m.std(),iter_m.mean().iter_m.std()

## Image Denoising

In [ ]:
def read_and_split_png(image_path):

    img = Image.open(image_path)
    img = img.convert("RGB").resize((128,128))
    img_array = np.array(img,dtype=float)
    R, G, B = img_array[:, :, 0], img_array[:, :, 1], img_array[:, :, 2]
    return R, G, B

def merge_and_save_png(R, G, B, save_path):
    merged_img_array = np.stack((R, G, B), axis=-1)
    merged_img = Image.fromarray(merged_img_array.astype("uint8"), "RGB")
    merged_img.save(save_path)

input_path = "star.png"  # 输入图像路径
output_path = "png/noise_star.png"  # 输出图像路径

R, G, B = read_and_split_png(input_path)
cols,rows = R.shape

R = add_gaussian_noise(R,0,std = 1)
G = add_gaussian_noise(G,0,std = 1)
B = add_gaussian_noise(B,0,std = 1)


merge_and_save_png(R, G, B, output_path)
k=0
BB=1
l = 10
rho = 1
R1 = dogeADMM(R,k=k,B=BB,l=l,rho=rho,max_iter=50)
G1 = dogeADMM(G,k=k,B=BB,l=l,rho=rho,max_iter=50)
B1 = dogeADMM(B,k=k,B=BB,l=l,rho=rho,max_iter=50)

merge_and_save_png(R1.reshape(cols,rows), G1.reshape(cols,rows), B1.reshape(cols,rows), f'png/filtered_star.png')